# Talktorial 12 (WIP)

# Binding Site Detection (DoGSiteScorer)

Abishek Laxmanan Ravi Shankar, Volkamer Lab

**NOTE: This notebook is work in progress. We plan to include parts of this notebook in the new round of talktorials on structural bioinformatics for the [TeachOpenCADD platform](https://github.com/volkamerlab/TeachOpenCADD).**

## Aim of the talktorial

In this talktorial, we **detect the ligand binding site** of a protein structure at the example of **EGFR** using the **DoGSiteScorer** protein.plus web server. Then, we **compare the results** with the binding pocket defined by KLIFS using the **KLIFS API**. The comparison is performed by calculating the coverage percentage of DoGSiteScorer in accordance with the KLIFS results. 

<div style="width:500; font-size:100%; text-align:center;">
    <img src="images/Figure1_Work_flow.jpg" alt="alternate text"  width="500" style="padding:0.8em;"/>
    Figure 1: Workflow of the talktorial - Detection of binding site with DoGSiteScorer and comparison with KLIFS binding site (Figure created by Abishek Laxmanan Ravi Shankar)
</div>

# Learning goals

### Theory
* Protein binding sites
* Binding site detection 
    * Introduction
        * Geometry-based approaches
        * Energy-based approaches
    * DogSiteScorer    
        * Pocket detection
        * Druggability
* Comparison
    * KLIFS pocket definitions

### Practical

* Binding site detection using DoGSiteScorer
    * Submit job to DoGSiteScorer and get job location
    * Get DoGSiteScorer pocket metadata
    * Sort pockets based on their drugScore
    * Get best binding site file content
    * Get residue IDs and names of best pocket
* Get KLIFS pocket
    * Get KLIFS structure ID for a PDB ID of interest
    * Get pocket from KLIFS ID and extract residues
* Comparison DoGSiteScorer and KLIFS pocket
    * Get DoGSiteScorer coverage with KLIFS results



## References

* Combining global and local measures for structure-based druggability predictions.
([<i>Noordwijkerhout Cheminformatics</i> 2011, <b>52</b>, 360-372](https://www.ncbi.nlm.nih.gov/pubmed/22148551))

* Prediction, Analysis, and Comparison of Active Sites 
([ <i>Journal of Chemical Information and Modeling</i> 2018 <b>6,7</b>](https://onlinelibrary.wiley.com/doi/abs/10.1002/9783527806539.ch6g))

* DoGSiteScorer
([<i>Bioinformatics</i> 2012, <b>28</b>, 2074-2075](https://doi.org/10.1093/bioinformatics/bts310)) 

* Protein–ligand complex
([Protein complex wiki](https://en.wikipedia.org/wiki/Protein%E2%80%93ligand_complex))

* Voronoi tessellation
([Voronoi tessellation Wiki](https://en.wikipedia.org/wiki/Voronoi_diagram))

* KLIFS database
( [KLIFS Website](https://klifs.net/))  

* Fragments based drug Discovery
([Google books](https://books.google.de/books?id=tjfvCQAAQBAJ&pg=PA202&lpg=PA202&dq=Fragments+based+drug+Discovery+SURFNET&source=bl&ots=O6XVGMVPTV&sig=ACfU3U2K6btUeMpgN4FBe627jpIJL6-mtQ&hl=en&sa=X&ved=2ahUKEwjonezYmbLkAhXBRxUIHdzKClIQ6AEwAnoECAkQAQ))

* KLIFS: a structural kinase-ligand interaction database
([<i>KLIFS Pubmed</i> 2016, Nucleic Acids Research, <b>44</b>, 365-371](https://www.ncbi.nlm.nih.gov/pubmed/26496949)) 

* KLIFS: A Knowledge-Based Structural Database To Navigate Kinase−Ligand Interaction Space
([<i>KLIFS Journal of Medicinal Chemistry</i> 2013, <b>57</b>, 249-257](https://pubs.acs.org/doi/10.1021/jm400378w))

* SURFNET
([<i>Journal of Molecular Graphics</i>1995, <b>13</b>323-330](https://www.ncbi.nlm.nih.gov/pubmed/8603061))

* LIGSITE
([<i>Journal of Molecular Graphics and Modelling</i> 1997 <b>6</b> 359-363](https://www.sciencedirect.com/science/article/pii/S1093326398000023?via%3Dihub))

* Docking 
([Docking wiki](https://en.wikipedia.org/wiki/Docking_(molecular)))

* DrugSite
([<i>Genome Informatics</i>2004 <b>15</b> 31–41 ](https://pdfs.semanticscholar.org/59ca/aac9a79c29660385af0ab83b82fba57cc476.pdf))

* Metapocket 2.0
([Summary of various binding site detection approaches](https://projects.biotec.tu-dresden.de/metapocket/algorithm.php))


## Theory

### Protein binding sites

The main objective of pharmaceutical research is to find a new drug inorder to cure a specific disease. Structure-based approaches in drug design develop drugs with respect to a target protein which is associated with the disease under investigation. However, in order to develop such a drug it is crucial to know where the drug is supposed to bind in the protein and therefore inhibit or activate the protein. Not always such a small molecule binding site is known. Therefore it is important to have tools for binding site detection.

Binding site detection has a special mention in the process of drug development. Binding sites are defined as a **hollow 3 dimensional space** on the surface of a protein molecule, which serves as the region for **ligand**, **peptide**, or **protein** binding. There are various factors which influence whether such a region is suitable binding site.
* Shape
* Spatial arrangement of the residues
* Interactions
    * Van der Waals interactions
    * Hydrogen bond interactions
    * Hydrophobic regions
    * Electrostatic interactions
    * π-π interactions

### Binding site detection 

### Introduction 

If ligand information is available (protein-ligand complex), then the ligand-surrounding protein region is defined as pocket (e.g. using a 6 Å radius).
If the ligand is absent, detection tools can be used for *in silico* pocket detection. 
These methods can be grouped into standard approaches, i.e. **geometry-based** and **energy-based** approaches as well as **grid-based** and **grid-free** approaches as outlined in the Figure 1.

<img src="images/Figure2_DetectionMethods.png" alt="alternate text"  width="500" style="padding:0.8em;"/>

Figure 2: Binding site detection methods can be grouped into geometry-based and energy-based approaches as well as grid-based and grid-free approaches. Figure from Prediction, Analysis, and Comparison of Active Sites. Volkamer et al., 2018 ([<i>Journal of Chemical Information and Modeling</i> 2018 <b>6,7</b>](https://onlinelibrary.wiley.com/doi/abs/10.1002/9783527806539.ch6g))

#### Geometry-based approaches

Geometry-based approaches analyze the shape of a molecular surface to locate cavities. 
They are based upon the 3D spatial arrangement of the atoms on the protein surface. 
They are broadly divided into two namely grid-based and grid-free approaches (checking the environment per grid point or without any grid, respectively).

* **Grid-based approaches**

    * Examples: POCKET, LIGSITE and DoGSiteScorer 
    * In LIGSITE, the cartesian coordinates runs along the X, Y and Z axes through a 3D region which consists of protein surfaces on both sides.
    * In addition to these axes a cubic diagonal passes along to find out the width of the cleft in between the protein surface. Therefore LIGSITE checks in seven directions the distance of each atom in the protein surface.
    * Therefore these coordinates enables LIGSITE to differentiate the Protein Solvent Protein regions (PSP) on the protein surface.
    * DoGSiteScorer is explained in detail below.
   
* **Grid-free approaches**

    * Example: SURFNET
    * In SURFNET, spheres are placed on the protein surface instead of having grids. 
    * The probe spheres are placed midway between the pairs of atoms on the protein. In case a probe overlaps any nearby atom, its radius is reduced until no overlap occurs. 
    * The resulting probes define the pocket and the cavities. The cutoff radius of the spheres are of 1 to 4 Angstrom.

#### Energy-based approaches

Interactions of a molecular fragment with protein are recorded. Favorable energetic responses are assigned to pockets. DrugSite and Docking are the well known energy-based methods. They are again divided into grid-based and grid-free approaches.

* **Grid-based approaches**

    * Example: DrugSite
    * This method uses carbon probes placed on each grid point, and van der Waal's energies between the probe and the protein with distance less than 8 Å are calculated. Mean energy and standard deviation are calculated for these grid points. 
    * Grid points with unfavorable energies are removed. Grid points fulfilling these cut-off are merged to pockets.

* **Grid-free approaches**

    * Example: Docking
    * The docking process is similar to the "lock and key" model in which the **target protein resembles** the **lock** and **ligand** to the **key**.
    * In this method, a scoring function is incorporated to evaluate the fragments that are placed at a position on the protein surface.
    * Scoring functions are physics based, which uses Molecular Mechanics force fields, which primarily estimates the amount of affinity between the binding ligand and the protein.
    * The scoring functions calculates the feasibility of whether the ligand could be docked to the particular protein.
    * These pockets are assigned based on the quantity of fragments that bind to a specific area.

### DogSiteScorer 

Certain geometric and physiochemical properties like pocket depth, pocket volume and surface are calculated in an automatic manner for the predicted pockets and subpockets. In **DogSiteScorer**, pockets and subpockets are predicted with DoGSite using a **Difference of Gaussian** filter. DogSiteScorer falls under the category of geometry-based and grid-based approaches. Pocket volume and surface are calculated by counting the **grid points** constituting the pocket volume or its surface and multiplying this number with the grid box volume or surface, respectively. A **breadth-first search** algorithm is used for pocket depth computation, starting from the solvent exposed pocket parts towards the most deeply buried regions. 

#### Pocket Detection

* The protein structure is placed in a grid of coordinates.
* **Volume** of the pocket is defined as a product of 'volume grid points' and 'cubed grid spacing'.
* **Depth** is defined as 'the maximum possible distance between the solvent and the buried part of the pocket'.
* **Difference of Gaussian** filter is applied across the edges of the grid.
* This operation helps to find the position on a protein surface where the location of a **sphere-like object** is favorable.
* The last step in this algorithm is the merging of neighboring subpockets to pockets.

<img src="images/Figure3_BindingSitePocket.png" alt="alternate text"  width="400" style="padding:0.8em;"/>
    
Figure 3: The binding site on the surface of the protein shown with grid view consisting of the surface molecules, volume inside the binding region, and the region exposed to the exterior of the protein 
Figure from "Combining global and local measures for structure-based druggability predictions."
([<i>Noordwijkerhout Cheminformatics</i> 2011, <b>52</b>, 360-372](https://www.ncbi.nlm.nih.gov/pubmed/22148551))


#### Druggability 

* The Druggability is based on Machine learning technique - **Support Vector Machine (SVM)**.
* Two strategies were used for the Druggability prediction, which includes, **Global machine learning approach** and **Nearest neighbour approach**. 
* **Discriminative Analysis** - Best suited to seperate druggable from undruggable.
* The model has been trained and tested on both the redundant and non-redundant version of the druggable dataset
* This model showed a mean accuracy of 90%.
* The druggabilty score lies between 0 and 1. The higher the score the better the chance that the pocket is druggable.

<img src="images/Figure4_DogSiteAlgo.png" alt="alternate text"  width="1000" style="padding:0.8em;"/>
    
Figure 4: Flowsheet explaining the pipeline of DoGSiteScorer
Figure from "Combining global and local measures for structure-based druggability predictions."
([<i>Noordwijkerhout Cheminformatics</i> 2011, <b>52</b>, 360-372](https://www.ncbi.nlm.nih.gov/pubmed/22148551))

### Comparison

Once we obtain the best binding site from the DoGSiteScorer, we can compare the results with any other method in order to validate it. Here we compare it with the KLIFS binding pocket using the KLIFS API. 

#### KLIFS pocket definition

KLIFS stands for **Kinase-Ligand Interaction Fingerprints and Structures database**. It is a structural repository of over 2900 human and mouse kinase enzymes along with their catalytic activity. It also deals with the kinase domains and also the inhibitors which can interact with them. Kinases share a similar conserved regions, which serves as a challenge for small molecules that can selectively bind to a specific kinase. 

KLIFS enables us to do systematic comparison and analysis of chemical features of all available kinases in the process of ligand binding. The classification helps us of an all-encompassing binding site of **85 residues**. It is possible to compare the **interaction patterns** of **kinase-inhibitors** to each other to, for example, identify crucial interactions determining kinase-inhibitor selectivity. KLIFS API could return the binding pocket of a specific kinase protein. 

This data resource can therefore be used to compare the binding site with regard to DogSiteScorer for the **EGFR kinase**. 

## Practical

In [1]:
import time

from biopandas.mol2 import PandasMol2
from biopandas.pdb import PandasPdb
import pandas as pd
import requests

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Binding site detection using DoGSiteScorer

#### Submit job to DoGSiteScorer and get job location

In [3]:
def dogsitescorer_submit_with_pdbid(pdb_code, chain_id, ligand=''):
    """
    Submit PDB ID to DoGSiteScorer webserver using their API and get back URL for job location.
    
    Parameters
    ----------
    pdb_id : str 
        4-letter valid PDB ID, e.g. '3w32'.
    chain : str
        Chain ID, e.g. 'A'.
    ligand : str
        Name of ligand bound to PDB structure with pdb_id, e.g. 'W32_A_1101'. 
        Currently, the ligand name must be checked manually on the DoGSiteScorer website. 
        
    Returns
    -------
    str
        Job location URL for submitted query.
        
    References
    ----------
        Function is adapted from: https://github.com/volkamerlab/TeachOpenCADD/pull/3 (@jaimergp)
    """
    
    # Submit job to proteins.plus
    r = requests.post("https://proteins.plus/api/dogsite_rest",
        json={
            "dogsite": {
                "pdbCode": pdb_code,
                "analysisDetail": "1",
                "bindingSitePredictionGranularity": "1",
                "ligand": ligand,
                "chain": chain_id
            }
        },
        headers= {'Content-type': 'application/json', 'Accept': 'application/json'}
    )

    r.raise_for_status()
    
    return r.json()['location']

In [4]:
# Identifying the job location where the work is submitted to the web server
job_location = dogsitescorer_submit_with_pdbid('3w32', 'A')
job_location

'https://proteins.plus/api/dogsite_rest/A7bq49hpfbHjwGfr3GivXidr'

In [5]:
# Wait a bit so that job can finish
time.sleep(30)

#### Get DoGSiteScorer pocket metadata

In [6]:
def get_dogsitescorer_metadata(job_location):
    """
    Get results from a DoGSiteScorer query, i.e. the binding sites which are found over the protein surface, 
    in the form of a table with the details about all detected pockets.

    Parameters
    ----------
    job_location : str
        Consists of the location of a finished DoGSiteScorer job on the proteins.plus web server.
    
    Returns
    -------
    pandas.DataFrame
        Table with metadata on detected binding sites. 
    """
    
    # Get job results
    result = requests.get(job_location)
    
    # Get URL of result table file
    result_file = result.json()['result_table']
    
    # Get result table
    result_table = requests.get(result_file).text
    
    # Split string into list of lists (=table)
    result_table_split = [i.split('\t') for i in result_table[:-1].split('\n')]

    # Remove spaces
    result_table_split = [[j.replace(' ', '') for j in i] for i in result_table_split]

    # Extract column names, index names, table body
    column_names = result_table_split[0]
    index_names = [i[0] for i in result_table_split[1:]]
    table = [i[1:] for i in result_table_split[1:]]

    # Convert to DataFrame
    result_table_df = pd.DataFrame(
        table,
        columns=column_names[1:],
        index=index_names
    )
    result_table_df.index.name = 'name'
    
    # Convert number strings to numeric values
    for name, data in result_table_df.iteritems():
        try:
            result_table_df[name] = pd.to_numeric(data)
        except ValueError:
            pass
    
    return result_table_df

In [7]:
# Retrieving all the guessed pockets from DoGSiteScorer Web server 
metadata = get_dogsitescorer_metadata(job_location)
metadata

,lig_cov,poc_cov,lig_name,volume,enclosure,surface,depth,surf/vol,lid/hull,ellVol,ellc/a,ellb/a,siteAtms,accept,donor,hydrophobic_interactions,hydrophobicity,metal,Cs,Ns,Os,Ss,Xs,negAA,posAA,polarAA,apolarAA,ALA,ARG,ASN,ASP,CYS,GLN,GLU,GLY,HIS,ILE,LEU,LYS,MET,PHE,PRO,SER,THR,TRP,TYR,VAL,simpleScore,drugScore
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
P_0,0.0,0.0,"""""",1422.66,0.10,1673.75,19.26,1.176493,-,-,0.13,0.67,288,86,40,71,0.36,0,198,45,41,4,0,0.10,0.13,0.24,0.53,4,5,2,5,2,2,1,5,0,3,12,3,2,3,3,1,2,1,1,5,0.63,0.810023
P_0_0,0.0,0.0,"""""",599.23,0.06,540.06,17.51,0.901257,-,-,0.14,0.22,131,35,13,25,0.34,0,95,16,17,3,0,0.03,0.10,0.28,0.59,1,2,1,1,2,1,0,2,0,2,7,1,2,2,1,0,2,0,0,2,0.59,0.620201
P_0_1,0.0,0.0,"""""",201.73,0.08,381.07,11.36,1.889010,-,-,0.17,0.25,51,17,9,10,0.28,0,36,6,7,2,0,0.08,0.17,0.25,0.50,1,1,0,1,1,1,0,0,0,0,3,1,1,0,0,0,1,0,0,1,0.17,0.174816
P_0_2,0.0,0.0,"""""",185.60,0.17,282.00,9.35,1.519397,-,-,0.45,0.55,48,17,8,12,0.32,0,31,8,8,1,0,0.17,0.25,0.08,0.50,0,2,0,1,0,0,1,1,0,0,2,1,1,1,0,0,0,0,0,2,0.13,0.195695
P_0_3,0.0,0.0,"""""",175.30,0.15,297.42,9.29,1.696634,-,-,0.23,0.37,48,16,8,14,0.37,0,32,8,8,0,0,0.14,0.14,0.36,0.36,1,1,1,2,0,0,0,3,0,0,1,1,0,1,1,1,0,0,0,1,0.13,0.168845
P_0_4,0.0,0.0,"""""",170.37,0.08,390.10,11.99,2.289722,-,-,0.16,0.20,47,14,7,17,0.45,0,34,7,6,0,0,0.00,0.18,0.18,0.64,2,2,0,0,0,1,0,0,0,1,3,0,0,0,0,0,0,1,1,0,0.15,0.223742
P_0_5,0.0,0.0,"""""",90.43,0.24,177.50,6.24,1.962844,-,-,0.70,0.89,26,8,6,5,0.26,0,16,6,4,0,0,0.12,0.25,0.25,0.38,0,1,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,2,0.00,0.165232
P_1,0.0,0.0,"""""",708.99,0.13,1030.19,14.32,1.453039,-,-,0.14,0.59,140,44,13,34,0.37,0,98,17,25,0,0,0.14,0.11,0.36,0.39,3,1,1,0,0,0,4,4,0,1,4,2,0,1,1,2,2,0,1,1,0.46,0.755915
P_1_0,0.0,0.0,"""""",496.90,0.11,739.17,12.72,1.487563,-,-,0.14,0.18,103,34,8,22,0.34,0,74,12,17,0,0,0.18,0.09,0.32,0.41,2,1,1,0,0,0,4,1,0,1,4,1,0,0,1,2,2,0,1,1,0.49,0.465489


#### Sort pockets based on their drugScore

In [8]:
# Sort the obtained binding site by descending drugScore
metadata.sort_values(by='drugScore', ascending=False)

,lig_cov,poc_cov,lig_name,volume,enclosure,surface,depth,surf/vol,lid/hull,ellVol,ellc/a,ellb/a,siteAtms,accept,donor,hydrophobic_interactions,hydrophobicity,metal,Cs,Ns,Os,Ss,Xs,negAA,posAA,polarAA,apolarAA,ALA,ARG,ASN,ASP,CYS,GLN,GLU,GLY,HIS,ILE,LEU,LYS,MET,PHE,PRO,SER,THR,TRP,TYR,VAL,simpleScore,drugScore
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
P_0,0.0,0.0,"""""",1422.66,0.10,1673.75,19.26,1.176493,-,-,0.13,0.67,288,86,40,71,0.36,0,198,45,41,4,0,0.10,0.13,0.24,0.53,4,5,2,5,2,2,1,5,0,3,12,3,2,3,3,1,2,1,1,5,0.63,0.810023
P_1,0.0,0.0,"""""",708.99,0.13,1030.19,14.32,1.453039,-,-,0.14,0.59,140,44,13,34,0.37,0,98,17,25,0,0,0.14,0.11,0.36,0.39,3,1,1,0,0,0,4,4,0,1,4,2,0,1,1,2,2,0,1,1,0.46,0.755915
P_0_0,0.0,0.0,"""""",599.23,0.06,540.06,17.51,0.901257,-,-,0.14,0.22,131,35,13,25,0.34,0,95,16,17,3,0,0.03,0.10,0.28,0.59,1,2,1,1,2,1,0,2,0,2,7,1,2,2,1,0,2,0,0,2,0.59,0.620201
P_3,0.0,0.0,"""""",244.16,0.04,514.94,14.34,2.109027,-,-,0.12,0.31,92,18,7,24,0.49,0,67,12,13,0,0,0.12,0.06,0.41,0.41,0,0,1,1,1,0,1,1,1,0,1,0,0,2,3,2,1,0,1,1,0.12,0.572013
P_4_1,0.0,0.0,"""""",60.93,0.20,193.96,5.80,3.183325,-,-,0.54,0.84,28,4,1,16,0.76,0,22,4,2,0,0,0.00,0.14,0.14,0.71,0,0,0,0,0,0,0,0,0,1,0,1,1,1,2,0,1,0,0,0,0.00,0.570708
P_2,0.0,0.0,"""""",286.21,0.18,462.29,11.83,1.615213,-,-,0.23,0.44,70,20,9,13,0.31,0,50,9,9,2,0,0.12,0.06,0.18,0.65,1,0,0,2,0,1,0,0,0,1,1,1,2,1,2,0,1,0,1,3,0.09,0.537137
P_1_0,0.0,0.0,"""""",496.90,0.11,739.17,12.72,1.487563,-,-,0.14,0.18,103,34,8,22,0.34,0,74,12,17,0,0,0.18,0.09,0.32,0.41,2,1,1,0,0,0,4,1,0,1,4,1,0,0,1,2,2,0,1,1,0.49,0.465489
P_4,0.0,0.0,"""""",169.28,0.21,373.47,11.49,2.206226,-,-,0.12,0.16,59,14,3,24,0.59,0,44,7,7,1,0,0.08,0.08,0.17,0.67,0,0,0,1,0,1,0,0,0,3,0,1,2,1,2,0,1,0,0,0,0.07,0.424397
P_5,0.0,0.0,"""""",166.59,0.16,347.46,11.99,2.085719,-,-,0.14,0.24,58,18,8,10,0.28,0,39,8,11,0,0,0.15,0.08,0.46,0.31,0,0,1,1,0,2,1,1,1,1,1,0,1,0,1,1,0,0,1,0,0.00,0.401384


In [9]:
def select_best_pocket(metadata, by='drugScore'):
    """
    select_best_pocket - This function uses 'drugScore' as a parameter to identify the best pocket 
    among the obtained pockets.
    
    Parameters
    ----------
    metadata : pd.DataFrame
        Guessed pockets retrieved from the DoGSiteScorer website
        
    by : str
        Method name to sort table by (default is to sort by drugScore).
        
    Returns 
    -------
    str
        Best binding site name.
    """
    
    by_methods = ['drugScore', 'volume']
    
    # Sort by best druggability score
    if by == 'drugScore':
        sorted_pocket = metadata.sort_values(by='drugScore', ascending=False)
    # Sort by volume
    elif by == 'volume':
        sorted_pocket = metadata.sort_values(by='volume', ascending=False)
    else:
        raise ValueError(f'Selection method not in list: {", ".join(by_methods)}')
                         
    # Get name of best pocket
    best_pocket_name = sorted_pocket.iloc[0, :].name     
        
    return best_pocket_name

In [10]:
# Get the name of the best pocket
best_pocket = select_best_pocket(metadata, by='drugScore')
best_pocket

'P_0'

#### Get best binding site file content

In [11]:
def get_pocket_locations(job_location):
    """
    Get the all pocket file locations for a finished DoGSiteScorer job.
    
    Parameters
    ----------
    job_location : str
        URL of finished job submitted to the DoGSiteScorer web server.
    
    Returns
    -------
    list
        List of all pocket file location on the DoGSiteScorer web server.
    """
    
    # Get job results
    result = requests.get(job_location)
    
    # Get residues
    return result.json()['residues']

In [12]:
def get_best_pocket_location(pocket_files, best_pocket):
    """
    Get the best binding site file location.
    
    Parameters
    ----------
    pocket_files : list
        List of all pocket file location on the DoGSiteScorer web server.
    best_pocket : str
        Best binding site name.

    Returns
    ------
    str
        Best pocket file location on the DoGSiteScorer web server.
    """
    result = []
    
    for pocket_file in pocket_files:
        
        if f'{best_pocket}_res' in pocket_file:
            result.append(pocket_file)
            
    if len(result) > 1:
        raise TypeError(f'Multiple strings detected: {", ".join(result)}.')
    elif len(result) == 0:
        raise TypeError(f'No string detected.')
    else:
        pass
            
    return result[0]

In [13]:
# Get URL for all PDB files
pocket_files = get_pocket_locations(job_location)
pocket_files

['https://proteins.plus/results/dogsite/A7bq49hpfbHjwGfr3GivXidr/3w32_P_0_res.pdb',
 'https://proteins.plus/results/dogsite/A7bq49hpfbHjwGfr3GivXidr/3w32_P_0_0_res.pdb',
 'https://proteins.plus/results/dogsite/A7bq49hpfbHjwGfr3GivXidr/3w32_P_0_1_res.pdb',
 'https://proteins.plus/results/dogsite/A7bq49hpfbHjwGfr3GivXidr/3w32_P_0_2_res.pdb',
 'https://proteins.plus/results/dogsite/A7bq49hpfbHjwGfr3GivXidr/3w32_P_0_3_res.pdb',
 'https://proteins.plus/results/dogsite/A7bq49hpfbHjwGfr3GivXidr/3w32_P_0_4_res.pdb',
 'https://proteins.plus/results/dogsite/A7bq49hpfbHjwGfr3GivXidr/3w32_P_0_5_res.pdb',
 'https://proteins.plus/results/dogsite/A7bq49hpfbHjwGfr3GivXidr/3w32_P_1_res.pdb',
 'https://proteins.plus/results/dogsite/A7bq49hpfbHjwGfr3GivXidr/3w32_P_1_0_res.pdb',
 'https://proteins.plus/results/dogsite/A7bq49hpfbHjwGfr3GivXidr/3w32_P_1_1_res.pdb',
 'https://proteins.plus/results/dogsite/A7bq49hpfbHjwGfr3GivXidr/3w32_P_2_res.pdb',
 'https://proteins.plus/results/dogsite/A7bq49hpfbHjwGfr3Giv

In [14]:
# Get URL for PDB file containing the best pocket
best_pocket_location = get_best_pocket_location(pocket_files, best_pocket)
best_pocket_location

'https://proteins.plus/results/dogsite/A7bq49hpfbHjwGfr3GivXidr/3w32_P_0_res.pdb'

#### Get residue IDs and names of best pocket

In [15]:
def get_pocket_residues(pocket_location):
    """
    Gets residue IDs and names of a pocket.
    
    Parameters
    ----------
    pocket_location : str
        Best pocket file location on the DoGSiteScorer web server.
        
    Returns
    -------
    pandas.DataFrame
        Table of residues names and IDs for the best obtained binding site.
    """
    
    # Retrieve PDB file content from URL
    result = requests.get(pocket_location)

    # Get content of PDB file  
    pdb_residues = result.text
    
    # Load PDB format as DataFrame
    ppdb = PandasPdb()
    pdb_df = ppdb._construct_df(pdb_residues.splitlines(True))['ATOM']
    
    return pdb_df[['residue_number', 'residue_name']]

In [16]:
# Get residues of best pocket
pocket_residues = get_pocket_residues(best_pocket_location)
pocket_residues.head()

,residue_number,residue_name
0,701,GLN
1,701,GLN
2,701,GLN
3,701,GLN
4,702,ALA


### Get KLIFS pocket

We have identified the binding site of a kinase of interest with the help of DoGSiteScorer web server. 
Now we want to check if the obtained binding site is in agreement with the defined binding site defined by the KLIFS database. 

In [17]:
from bravado.client import SwaggerClient

KLIFS_API_DEFINITIONS = "https://klifs.net/swagger/swagger.json"
KLIFS_CLIENT = SwaggerClient.from_url(KLIFS_API_DEFINITIONS, config={'validate_responses': False})

#### Get KLIFS structure ID for a PDB ID of interest 

In [18]:
def get_klifsid_from_pdbid(pdb_codes):
    """
    Gets KLIFS structure ID for a PDB ID of interest.
    
    Parameters
    ----------
    pdb_codes : str or list of str
        PDB ID of our desired kinase protein.
    
    Returns
    -------
    int
        KLIFS ID value of the given PDB ID.
    """
    
    # If only a single PDB ID is provided in the form of a string, cast string to list
    if isinstance(pdb_codes, str):
        pdb_codes = [pdb_codes]
    
    # Get metadata for KLIFS structures associated with one or more PDB IDs
    klifs_structures = KLIFS_CLIENT.Structures.get_structures_pdb_list(
        pdb_codes=pdb_codes
    ).response().result
    
    # Get KLIFS IDs for KLIFS structures
    klifs_ids = [i.structure_ID for i in klifs_structures]
    
    # Get first KLIFS ID
    klifs_id = klifs_ids[0]
    
    return int(klifs_id)

In [19]:
# Get KLIFS ID
klifs_id = get_klifsid_from_pdbid('3w32')
klifs_id

784

#### Get pocket from KLIFS ID and extract residues

In [20]:
def get_klifs_pocket(klifs_id):
    """
    Get binding site residues by KLIFS ID.
    
    Parameters
    ----------
    klifs_id : str
        ID which is provided by the KLIFS database when the PDB ID of our desired protein is given.
    
    Returns
    -------
    str
        Mol2 file content from KLIFS database.
    """
    
    klifs_pocket = KLIFS_CLIENT.Structures.get_structure_get_pocket(
        structure_ID=klifs_id
    ).response().result
    
    return klifs_pocket

In [21]:
def get_klifs_residues(klifs_pocket):
    """
    Get KLIFS residue names and IDs.
    
    Parameters
    ----------
    klifs_pocket : str
       Mol2 file content from KLIFS database.
    
    Returns
    -------
    pandas.DataFrame
        Table of mol2 file residue names and IDs.
    """
    
    ppdb = PandasMol2()
    mol2_df = ppdb._construct_df(
        klifs_pocket.splitlines(True),  
        #TODO: Rename 'Residues' to 'subst_name'
        col_names=['atom_id','atom_name','x','y','z','atom_type','subst_id','Residues','charge','backbone'],
        col_types=[int, str, float, float, float, str, int, str, float, str]

    )
    
    # Split residue ID and name (in order to have the same output as for the DSS pocket residues)
    klifs_residues = mol2_df[['Residues']].Residues

    klifs_residue_name = list(klifs_residues.apply(lambda x: x[:3]))
    klifs_residue_id = list(klifs_residues.apply(lambda y: int(y[3:])))

    klifs_residues = pd.DataFrame(
        [klifs_residue_id, klifs_residue_name], 
        index=['residue_number', 'residue_name']
    ).transpose()
    
    return klifs_residues

In [22]:
# Get KLIFS pocket
klifs_pocket = get_klifs_pocket(klifs_id)
klifs_pocket[:1000]

'@<TRIPOS>MOLECULE\nHUMAN/EGFR_3w32_chainA\n1364 1368 85 0 0 \nBIOPOLYMER\nUSER_CHARGES\n\n\n@<TRIPOS>ATOM\n   1 N        7.7187    15.4685    51.5111 N.3    1 LYS716   0.0000 BACKBONE\n   2 H        7.3300    16.1756    50.9036 H      1 LYS716   0.0000 BACKBONE\n   3 CA       7.3282    14.0740    51.3137 C.3    1 LYS716   0.0000 BACKBONE\n   4 HA       8.1219    13.4452    51.7172 H      1 LYS716   0.0000 BACKBONE\n   5 C        7.1509    13.7803    49.8284 C.2    1 LYS716   0.0000 BACKBONE\n   6 O        6.8337    14.6724    49.0417 O.2    1 LYS716   0.0000 BACKBONE\n   7 CB       6.0216    13.7553    52.0569 C.3    1 LYS716   0.0000 \n   8 HB2      5.6643    12.7873    51.7056 H      1 LYS716   0.0000 \n   9 HB3      5.2944    14.5258    51.8006 H      1 LYS716   0.0000 \n  10 CG       6.1220    13.6938    53.5737 C.3    1 LYS716   0.0000 \n  11 HG2      5.1238    13.8138    53.9949 H      1 LYS716   0.0000 \n  12 HG3      6.7606    14.5086    53.9147 H      1 LYS716   0.0000 \n  13

In [23]:
# Get KLIFS residues
klifs_residues = get_klifs_residues(klifs_pocket)
klifs_residues.head()

,residue_number,residue_name
0,716,LYS
1,716,LYS
2,716,LYS
3,716,LYS
4,716,LYS


### Comparison of DoGSiteScorer and KLIFS pocket

Here we compare the above two results by calculating the set intersection between the residue IDs obtained from DoGSiteScorer and KLIFS and then calculating the percentage of coverage over the KLIFS residue IDs.  

In [24]:
len(set(klifs_residues.residue_number))

85

In [25]:
len(set(pocket_residues.residue_number))

62

####  Get DoGSiteScorer coverage with KLIFS results

In [26]:
def get_DSS_coverage(pocket_residues, klifs_residues):
    """
    Get the percentage of coverage when the output from DoGSiteScorer website is 
    compared with the KLIFS database residues.
    
    Parameters
    ----------
    pocket_residues : Pandas dataFrame
        Consists of binding site residues returned from DoGSiteScorer website
    klifs_residues : Pandas datatype
        Consists of binding site residues returned from KLIFS API
    
    Returns
    -------
    float
        Percentage of DSS residue coverage compared to KLIFS pocket.   
    """

    pocket_residues_set = set(pocket_residues.residue_number)
    klifs_residues_set = set(klifs_residues.residue_number)
    
    # Number of equal residue IDs in both pockets
    set_intersection = len(pocket_residues_set.intersection(klifs_residues_set))
    print(f'Number of shared pocket residues: {set_intersection}')
    
    # Number of total residues in both pockets 
    set_union = len(pocket_residues_set.union(klifs_residues_set))
    print(f'Number of total pocket residues: {set_union}')
    
    # Get percentage of DSS pocket residues that are listed in KLIFS
    DSS_coverage = set_intersection / len(pocket_residues_set) * 100
    
    return DSS_coverage

In [27]:
# Final output percentage
DSS_coverage = get_DSS_coverage(pocket_residues, klifs_residues)
DSS_coverage

Number of shared pocket residues: 45
Number of total pocket residues: 102


72.58064516129032

#### Result Discussion

When comparing the results of DoGSiteScorer with KLIFS API the percentage of **shared binding site residues** is "72.58064516129032".